In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix, balanced_accuracy_score
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler
from sklearn.decomposition import PCA
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
from tensorflow.keras.layers import Input, Dense, Attention
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
dataset_path = 'NVDA.csv' 
data = pd.read_csv(dataset_path)

X = data[['Open', 'High', 'Low']]
y = data['Close'] 

print(pd.DataFrame(X).head())

       Open      High       Low
0  0.424410  0.426933  0.415239
1  0.422347  0.434728  0.422347
2  0.429913  0.433811  0.425786
3  0.430601  0.432436  0.421200
4  0.420971  0.428308  0.418449


Abordagens:  
O número de neurônios escondidos deve estar entre o tamanho da camada de entrada e o da camada de saída.  
O número de neurônios escondidos deve ser 2/3 do tamanho da camada de entrada, mais o tamanho da camada de saída

In [25]:
def n_neuronios(X, y):
    n_neur = [0] * 3
    input_dim = X.shape[1]
    output_dim = y.shape[0]
    #Neurónios na camada escondida 1
    n_neur [0] = int((input_dim + output_dim)/2)
    #Neurónios na camada escondida 2
    n_neur [1] = int((input_dim + output_dim)/3)
    #Neurónios na camada escondida 3
    n_neur [2] = int((input_dim + output_dim)/4)
    return n_neur, output_dim

Função gaussiana para inicializar os pesos

In [21]:
def initialize_weights(shape, dtype=None):
    mean = 0.0
    stddev = 0.01
    return tf.constant(np.random.normal(loc=mean, scale=stddev, size=shape), dtype=dtype)

Ajustar o Learning Rate para metade do seu valor de 10 em 10 epochs

In [22]:
def scheduler(epoch, lr): 
    lr_scheduler = LearningRateScheduler(scheduler)
    if epoch > 0 and epoch % 10 == 0: 
        return lr / 2 
    return lr

Verificar se a validation loss (no fold de desenvolvimento) não melhora em 10 epochs consecutivos, e para o treino caso isso aconteça

In [23]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

In [30]:
def model(X, n_neur, output_dim):
    input_layer = tf.keras.layers.Input(shape=(X.shape[1], X.shape[2])) 
    dense1 = tf.keras.layers.Dense(n_neur[0], kernel_initializer=initialize_weights)(input_layer)
    act1 = tf.keras.layers.LeakyReLU(negative_slope=0.01)(dense1)
    dense2 = tf.keras.layers.Dense(n_neur[1], kernel_initializer=initialize_weights)(act1)
    act2 = tf.keras.layers.LeakyReLU(negative_slope=0.01)(dense2)
    
    # Camada de atenção
    attention_layer = tf.keras.layers.Attention()([dense2, dense2])
    
    flatten = tf.keras.layers.Flatten()(attention_layer)
    output_layer = tf.keras.layers.Dense(output_dim, activation='linear', kernel_initializer=initialize_weights)(flatten)
    
    model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer)
    return model

In [33]:
def train(X, y):
    adam_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    n_neur, output_dim = n_neuronios(X, y)
    model = model(X, n_neur, output_dim)
    model.compile(optimizer=adam_optimizer, loss='sparse_categorical_crossentropy', metrics=["accuracy"])
    model.fit(X_train, y_train, epochs=100, batch_size=128, validation_data=(X_dev, y_dev), callbacks=[lr_scheduler, early_stopping])
    loss, accuracy = model.evaluate(X_test, y_test)
    print(f"Loss: {loss}")
    print(f"Accuracy: {accuracy}")

Referências:  
https://medium.com/@zhonghong9998/attention-mechanisms-in-deep-learning-enhancing-model-performance-32a91006092a  
https://www.turing.com/kb/how-to-choose-an-activation-function-for-deep-learning  
https://medium.com/@gamelover9899/how-to-choose-the-right-activation-function-for-your-neural-network-843d8dc85806